<div style="background-color: #cfc ; padding: 20px; border-radius: 10px ; border: 2px solid green;">
<p>
<font size="+3">  <b> <center> NISQ Amplitude Estimation notebook</center></b></font>
</p>    
</div>

## Import QCWare libraries

In [1]:
import quasar
from qcware import forge
import numpy as np
from qcware.forge.qutils import create_qdot_circuit
from qcware.forge.montecarlo.nisqAE import make_schedule, run_schedule, run_unary, compute_mle

## The NISQ Amplitude Estimation algorithm

Let us provide a high level description of the NISQ Amplitude Estimation method.

We start with an initial quantum circuit $\mathcal{A}$, which we assume performs a general mapping of the form 
$\lvert 0 \rangle \mapsto cos(\theta) \lvert 0 \rangle \lvert bad \rangle + sin(\theta) \lvert 1 \rangle \lvert good \rangle$. 

Our goal is to estimate within some accuracy $\epsilon$ the probability of getting a good state, namely estimate the parameter $\theta$.

The main idea is that not only do we have the possibility to sample from this quantum circuit directly (that gives us outcome $\lvert 1 \rangle$ with probability $\sin^2(\theta)$), but we can create deeper quantum circuits by repeating this initial quantum circuit sequentially, so that sampling from all these quantum circuits with different depths and cleverly combining all the results, reduces the total number of samples. 

The way to create these deeper quantum circuits is by defining the iteration circuit as $S_\chi\mathcal{A}^\dagger S_0 \mathcal{A}$, where $S_{\chi}$ is a unitary operator that flips the phase of the good states and $S_0$ is a unitary operation theta flips the $\lvert 0 \rangle$ state. Then, the circuit corresponding to depth $D$ is a concatenation of the initial circuit and $D$ times of the iteration circuit. In many cases, the definition of the optimal iteration circuit can be automated, but one can also provide their own iteration circuit. 

The next important part of the quantum Amplitude Estimation method is to define a schedule of which circuits to sample from and how many times. In other words, we need to define a schedule, which is a list of pairs {($D_1$,$N_1$), ($D_2$,$N_2$),...($D_k$,$N_k$)}, that tells the quantum algorithm to run each quantum circuit of depth $D_i$ for $N_i$ shots. Different schedules give different accuracies with different number of samples! Here, we have some predefined types of schedules ('linear','exponential','powerlaw,'direct') one can use or one can define their own schedule by just providing a list.

Last, we perform a Maximum Likelihood Estimation to provide the most accurate value.

### Example 1

The simplest possible case: a circuit such that $\lvert 0 \rangle \mapsto cos(\theta) \lvert 0 \rangle + sin(\theta) \lvert 1 \rangle$ for some unknown $\theta$. The goal is to estimate $\theta$ within $\epsilon$.

#### Define the initial_circuit

In [2]:
# Example 1 : one qubit rotation

theta = np.random.rand(1)[0] * np.pi/2
print("Theta is: ", theta,"\n")

initial_circuit = quasar.Circuit().Rx(0,theta=theta)
print(initial_circuit)

Theta is:  0.049183277658301244 

T  : |0 |

q0 : -Rx-
         
T  : |0 |



#### Define a schedule

In [3]:
# Examples of schedules
# (schedule_type='direct', n_shots=10000) : [[0,10000]]
# (schedule_type='linear', max_depth=10,n_shots=10):  [[0,10],[1,10],[2,10],[3,10],[4,10],...,[10,10]]
# (schedule_type='exponential', max_depth=20,n_shots=10) [[1,10],[2,10],[4,10],[8,10],[16,10]]
# (schedule_type='powerlaw', beta=0.2, n_shots=10) : this is a more complicated schedule that depends on beta
# which takes values from [0.1 , 0.9]. Attention, the exponent beta can result in very large computations


schedule = None         # provide a schedule, eg [[0,20],[1,20],[2,20]] or put None for predefined schedule types
schedule_type = 'linear' # schedule type: 'direct', 'linear','exponential','powerlaw', (only if schedule=None)
n_shots = 10             # shots per circuit
max_depth = 20    # parameter for 'linear' and 'exponential' to define the maximum depth of a circuit
beta = 0.2              # parameter for the 'powerlaw' schedule between [0.1,0.9]

epsilon = 0.005 # the accuracy parameter

schedule = make_schedule(schedule_type=schedule_type, n_shots=n_shots, beta=beta, epsilon=epsilon)

In [4]:
schedule

[[0, 10],
 [1, 10],
 [2, 10],
 [3, 10],
 [4, 10],
 [5, 10],
 [6, 10],
 [7, 10],
 [8, 10],
 [9, 10],
 [10, 10],
 [11, 10],
 [12, 10],
 [13, 10],
 [14, 10],
 [15, 10],
 [16, 10],
 [17, 10],
 [18, 10],
 [19, 10],
 [20, 10]]

#### Get the samples for the quantum circuits

Here we use the function nisqAE_unary( ) which works for one-qubit circuits and circuits that only have support on unary states. This function takes as inputs only the initial_circuit and a schedule, computes the iteration circuit  internally and outputs an estimate. It uses the most NISQ circuits.

In [5]:
# we use the nisqAE_unary function that works for one qubit circuits or when the circuits only produce unary states
results = run_unary(initial_circuit, schedule)

In [6]:
results

[[[0, 10], 0],
 [[1, 10], 0],
 [[2, 10], 0],
 [[3, 10], 1],
 [[4, 10], 0],
 [[5, 10], 3],
 [[6, 10], 2],
 [[7, 10], 4],
 [[8, 10], 5],
 [[9, 10], 6],
 [[10, 10], 7],
 [[11, 10], 10],
 [[12, 10], 10],
 [[13, 10], 10],
 [[14, 10], 10],
 [[15, 10], 10],
 [[16, 10], 10],
 [[17, 10], 10],
 [[18, 10], 9],
 [[19, 10], 8],
 [[20, 10], 7]]

#### Post process them with MLE

In [7]:
# calculate_nisqAE_MLE(results,0.001)

# check if the value is close to the real one
print('NISQ AE estimate:', compute_mle(results,0.001))
print('Real value:', theta)
print('Estimation error:', np.abs(compute_mle(results,0.001) - theta) )

NISQ AE estimate: 0.05026548245743669
Real value: 0.049183277658301244


Estimation error: 0.0010822047991354486


### Example 2

We now use the qdot circuit that estimates the dot product between two input vectors. This is a unary quantum circuit so we can still use the nisqAE_unary( ) function.

In [8]:
dimension = 8

x = np.random.rand(dimension)
x = x/np.linalg.norm(x)
y = np.random.rand(dimension)
y = y/np.linalg.norm(y)

dot_circuit = create_qdot_circuit(x,y,loader_mode="parallel",absolute=True)
print(dot_circuit)

T  : |0|1|2|3|4|5|

q0 : -X-B-B-B-B-B-
        | | | | | 
q1 : ---|-|-S-|-|-
        | |   | | 
q2 : ---|-S-B-S-|-
        |   |   | 
q3 : ---|---S---|-
        |       | 
q4 : ---S-B-B-B-S-
          | | |   
q5 : -----|-S-|---
          |   |   
q6 : -----S-B-S---
            |     
q7 : -------S-----
                  
T  : |0|1|2|3|4|5|



In [9]:
schedule = make_schedule(schedule_type=schedule_type, n_shots=n_shots, beta=beta, epsilon= epsilon)
results2 = run_unary(dot_circuit, schedule)
estimated_theta = compute_mle(results2,0.001)

# compute the estimated dot product
estimated_dot= np.sin(estimated_theta)**2

# check if the value is close to the real one
print('NISQ AE estimate:', estimated_dot)
print('Real value:', np.dot(x,y)**2)
print('Estimation error:', np.abs(estimated_dot - np.dot(x,y)**2) )

NISQ AE estimate: 0.5454901019517849
Real value: 0.5442370682284103
Estimation error: 0.001253033723374597


### Example 3

We can also directly compute the iteration circuit ourselves and provide the target qubits and states to the more general function run_nisqAE_schedule( ). This is what we have done for pricing European Options using quantum Monte Carlo methods. 

Here we show it again for the one-qubit rotation circuit. 

In [10]:
# Example 1 : one qubit rotation

theta = np.random.rand(1)[0] * np.pi/2

initial_circuit = quasar.Circuit().Rx(0,theta=theta)
iteration_circuit = quasar.Circuit().Z(0).Rx(0,theta=-theta).Z(0).Rx(0,theta=theta)

schedule = make_schedule(schedule_type=schedule_type, n_shots=n_shots, beta=beta, epsilon= epsilon)
results3 = run_schedule(
    initial_circuit=initial_circuit, 
    iteration_circuit=iteration_circuit, 
    target_qubits=[0],
    target_states=[1],
    schedule=schedule)
estimated_theta = compute_mle(results3,0.001)

# check if the value is close to the real one
print('NISQ AE estimate:', estimated_theta)
print('Real value:', theta)
print('Estimation error:', np.abs(estimated_theta - theta) )

NISQ AE estimate: 1.4561281949388691
Real value: 1.4576963753686956
Estimation error: 0.0015681804298264268
